In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import nibabel as nib
import os
from sklearn.decomposition import PCA
import math
from skimage import io
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import time
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage.interpolation import rotate as rot
import glob
from scipy import ndimage
from scipy.ndimage.interpolation import zoom
import pickle

In [2]:
def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list

def l2norm(a,b):
    return np.sqrt(a**2+b**2)


def align_img(bv_label):
    x,y,z = np.where(bv_label==1)
    points = np.vstack((x,y,z))
    points = points.T
    #print(points.shape)
    pca = PCA(n_components = 3)
    pca.fit(points)
    pc1 = pca.components_[0,:]
    #print(pc1)
    pc2 = pca.components_[1,:]
    if pc1[0] >=0 and pc1[1] >=0:
        azimuth = -np.arctan(pc1[1]/pc1[0])*(360/(2*np.pi))
    #print('case1')
#     clockwise = 1

    elif pc1[0] <=0 and pc1[1] <=0:
        azimuth = (180 - np.arctan(pc1[1]/pc1[0])*(360/(2*np.pi)))
    #    print('case2')
    #     anticlockwise = 1

    elif pc1[0]>=0 and pc1[1]<=0:
        azimuth = np.arctan(np.abs(pc1[1])/np.abs(pc1[0]))*(360/(2*np.pi))
    #    print('case3')
    #     anticlockwise = 1
    else:
        azimuth = -(90 + np.arctan(np.abs(pc1[0])/np.abs(pc1[1]))*(360/(2*np.pi)))
    #    print('case4')
    #     clockwise = 1

    # finding the elevation angle
    if pc1[2]>=0:
        elevation = -np.arctan(pc1[2]/l2norm(pc1[0],pc1[1]))*(360/(2*np.pi))
     #   print('case1')
    else:
        elevation = np.arctan(np.abs(pc1[2])/l2norm(pc1[0],pc1[1]))*(360/(2*np.pi))
     #   print('case2')
   
    bv_label_copy = bv_label.copy()
    label_rot1 = rot(bv_label_copy,angle=azimuth,axes=(0,1))
    label_rot = rot(label_rot1,angle=elevation,axes=(0,2))
    
    xr,yr,zr = np.where(label_rot==1)
    points_rot = np.vstack((xr,yr,zr))
    points_rot = points_rot.T
    #print(points_rot.shape)
    pca_rot = PCA(n_components = 3)
    pca_rot.fit(points_rot)
    pc1r = pca_rot.components_[0,:]
    pc2r = pca_rot.components_[1,:]
    
    angle3 = np.arctan(pc2r[2]/pc2r[1])*(360/(2*np.pi))
    
    label_rot = rot(label_rot,angle=-angle3,axes=(1,2))
    
    return azimuth, elevation, angle3, label_rot

def save_img(img, label, count):
    img_nft = nib.Nifti1Image(img,np.eye(4))
    img_save_data_path = './resize_img/img{}.nii'.format(count)
    nib.save(img_nft,img_save_data_path)
    
    img_nft = nib.Nifti1Image(label,np.eye(4))
    img_save_data_path = './resize_img/label{}.nii'.format(count)
    nib.save(img_nft,img_save_data_path)
    

In [5]:
data_base_path = '/scratch/zq415/grammar_cor/Localization/data'
#data_folder_list = ['20180419_newdata_nii_with_filtered', 'new_data_20180522_nii']
data_folder_list2 = 'fix_organized_data_nii'

all_BVs = []
# for cur_floder in data_folder_list:
#     cur_folder_path = os.path.join(data_base_path,cur_floder)
#     all_BVs += glob.glob(cur_folder_path+'/*/*/*[Bb][Vv]*')
# print(len(all_BVs))

cur_folder_path = os.path.join(data_base_path,data_folder_list2)
all_BVs += glob.glob(cur_folder_path+'/*[Bb][Vv]*')
print(len(all_BVs))

all_data_list = []
same_name_num = 0
for full_bv_path in all_BVs:
    if 'BV' in full_bv_path:
        all_data_list.append((full_bv_path[:-14] + '.nii', full_bv_path))
    else:
        all_data_list.append((full_bv_path[:-9]+ '_2' + '.nii', full_bv_path))

196


In [6]:
all_data_list[188]

('/scratch/zq415/grammar_cor/Localization/data/fix_organized_data_nii/20180130_En1_E11p5_E2a_reg_2.nii',
 '/scratch/zq415/grammar_cor/Localization/data/fix_organized_data_nii/20180130_En1_E11p5_E2a_regprebv.nii')

In [8]:
all_data_list[556]

('/scratch/zq415/grammar_cor/Localization/data/fix_organized_data_nii/20180228_En1_E13p5_Ext_E6a_2.nii',
 '/scratch/zq415/grammar_cor/Localization/data/fix_organized_data_nii/20180228_En1_E13p5_Ext_E6aprebv.nii')

In [4]:
mutant_names = read_mutant_txt('mutant_imgs.txt')

In [5]:
count = 0
mutant_label = {}

for i,img_path in enumerate(all_data_list):
    img = nib.load(img_path[0])
    img = np.float32(img.get_data())
    
    img_label = nib.load(img_path[1])
    img_label = np.uint8(img_label.get_data())
    img_label[img_label>0] = 1
    
    print(np.shape(img),np.shape(img_label))
    
    azimuth, elevation, angle3, label_rot = align_img(img_label)
    
    img_rot1 = rot(img,angle=azimuth,axes=(0,1))
    img_rot2 = rot(img_rot1,angle=elevation,axes=(0,2))
    img_rot = rot(img_rot2,angle=-angle3,axes=(1,2))
    
    x_slice,y_slice,z_slice = ndimage.find_objects(label_rot)[0]
    print(count, 'bv size: ', x_slice.stop-x_slice.start, y_slice.stop-y_slice.start, z_slice.stop-z_slice.start)
    
    img_slice = img_rot[x_slice.start:x_slice.stop, y_slice.start:y_slice.stop, z_slice.start:z_slice.stop]
    label_slice = label_rot[x_slice.start:x_slice.stop, y_slice.start:y_slice.stop, z_slice.start:z_slice.stop]
    
    x, y, z = np.shape(label_slice)
    label_resized = zoom(label_slice, (112.0/x, 64.0/y, 64.0/z))
    label_resized[label_resized>=0.5] = 1
    label_resized[label_resized<0.5] = 0
    
    img_resized = zoom(img_slice, (112.0/x, 64.0/y, 64.0/z))
    
    save_img(img_resized, label_resized, count)
    count += 1
    
    if 'BV' in img_path[1]:
        bv_base_name = os.path.basename(img_path[0])[:-4]
    else:
        bv_base_name = os.path.basename(img_path[0])[:-6]
        
    if bv_base_name in mutant_names:
        mutant_label[i] = (i, 0, bv_base_name, label_resized, img_resized, img_path[1])
    else:
        mutant_label[i] = (i, 1, bv_base_name, label_resized, img_resized, img_path[1])
        


(210, 241, 261) (210, 241, 261)
0 bv size:  102 54 54
(210, 241, 241) (210, 241, 241)
1 bv size:  101 54 45
(210, 241, 261) (210, 241, 261)
2 bv size:  113 53 55
(210, 241, 241) (210, 241, 241)
3 bv size:  107 56 54


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(180, 241, 181) (180, 241, 181)
4 bv size:  106 57 59
(210, 241, 241) (210, 241, 241)
5 bv size:  106 54 55
(210, 241, 241) (210, 241, 241)
6 bv size:  106 57 48
(210, 241, 221) (210, 241, 221)
7 bv size:  106 57 48
(210, 241, 241) (210, 241, 241)
8 bv size:  107 53 53


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(210, 241, 261) (210, 241, 261)
9 bv size:  114 59 53
(210, 241, 241) (210, 241, 241)
10 bv size:  108 55 51
(180, 281, 181) (180, 281, 181)
11 bv size:  104 71 64
(180, 281, 201) (180, 281, 201)
12 bv size:  105 69 59
(180, 281, 161) (180, 281, 161)
13 bv size:  107 68 55


...

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(180, 201, 181) (180, 201, 181)
23 bv size:  83 54 42


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(180, 201, 161) (180, 201, 161)
24 bv size:  85 55 41
(180, 201, 161) (180, 201, 161)
25 bv size:  77 54 40
(180, 201, 161) (180, 201, 161)
26 bv size:  89 70 39
(180, 201, 181) (180, 201, 181)
27 bv size:  84 70 40
(180, 201, 141) (180, 201, 141)
28 bv size:  75 56 36
(180, 201, 141) (180, 201, 141)


KeyboardInterrupt: 

In [6]:
save_name = 'All_data_112_64_64.pickle'
save_file = open(os.path.join(os.getcwd(),'data',save_name),'wb')
pickle.dump(mutant_label,save_file)
save_file.close()


In [35]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
mutant_num = 0 
mutant_label = {}
test_mut = []
year_name = []
for i,img_path in enumerate(all_data_list):
    if 'BV' in img_path[1]:
        bv_base_name = os.path.basename(img_path[0])[:-4]
    else:
        bv_base_name = os.path.basename(img_path[0])[:-6]
        
    if bv_base_name in mutant_names:
        mutant_label[i] = (i, 0, bv_base_name, img_path[1], img_path[0])
        if '2016' in img_path[0]:
            test_mut.append(bv_base_name)
    else:
        mutant_label[i] = (i, 1, bv_base_name, img_path[1], img_path[0])
        
    if '2016' in img_path[0]:
        year_name.append(bv_base_name)
    

In [42]:
for i, name in enumerate(mutant_names):
    print(i, ': ',name)

mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]

group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])

0 :  20171208_En1_E10_E3a
1 :  20171211_En1_E10_E1a
2 :  20171211_En1_E10_E4b
3 :  20171002_En1_E11_E2a
4 :  20170619_En1_E12_Ex_E2a_reg
5 :  20170619_En1_E12_Ex_E4b_Mut_reg
6 :  20170619_En1_E12_E1a
7 :  20170619_En1_E12_E3a
8 :  20170705_En1_E12_E1a_filt
9 :  20170718_En1_E12_E7a_reg
10 :  20170718_En1_E12_E7b_reg
11 :  20171003_En1_E12_E4a_Mut
12 :  20170207_En1_E13_E3_Mut
13 :  20170207_En1_E13_E3c_Mut
14 :  20170207_En1_E13_E3d_Mut
15 :  20170207_En1_E13_E10d_Mut
16 :  20170207_En1_E13_E12a_Mut
17 :  20170207_En1_E13_E12b_Mut
18 :  20170207_En1_E13_E12c_Mut
19 :  20170207_En1_E13_E12d_Mut
20 :  20170706_En1_E13_E9a_Mut_reg
21 :  20170706_En1_E13_E10a-Mut_reg
22 :  20170706_En1_E13_E11a_Mut_reg
23 :  20171004_En1_E13_E5b_Mut
24 :  20171212_En1_E11_E3a
25 :  20171212_En1_E11_E4b
26 :  20171212_En1_E11_E6a
27 :  20171212_En1_E11_E7a
28 :  20171009_En1_E12_E10a
29 :  20171009_En1_E12_E11a
30 :  20171009_En1_E12_E12a
31 :  20171009_En1_E12_E4a
32 :  20171122_En1_E12p5_E1b
33 :  2017121

In [43]:
group_list

70

In [44]:
33 + len(mutant_group)

58